In [8]:
import os
import openai
import subprocess
from dotenv import load_dotenv
import json
import math
import pandas

def execute_python(file_path):
    # Run the Python file
    result = subprocess.run(['python', file_path], capture_output=True, text=True)

    # Check if the execution was successful
    if result.returncode == 0:
        # Get the output from the executed file
        output = result.stdout.strip()
        #print("Output:", output)
    else:
        # Print the error message if the execution failed
        output = result.stderr.strip()
        #print("Error:", error)
    return output

def get_response_from_gpt4(sys, usr, assist, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ]    
    
    )
    #res = (response['choices'][0]['message']['content'])
    return response


def get_response_from_gpt4_funct(sys, usr, assist, func, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4-0613", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    #model="gpt-3.5-turbo",
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ],
    functions = func
    )
    return response



querystr = """Obtain the data and build a fish capture monthly data frame containing years 1986 to 2012
The expected columns are: Date which is a datetime, 
Total (the total dollar captured) which is a number and Wheight representing the wheight of 
the capture which is a number. The Date column is the primary key for this data set. 
To solve this query you might have to construct the resulting dataframe by merging, appending and joining 
sepparate dataframes so the final number of years and the final number of columns might not be obtained 
in the first steap but rather in multiple ones. Do not create columns or any other data that doesnt exist 
at the moment.
"""


# key01 = """ Fish capture weekly data containing years 1986 to 2012
# The columns are: date in the format MM/dd/yyyy, 
# Total capture value which is a number and Wheight representing the wheight of 
# the capture whic is a number. Every row corresponds to a week data.
# """
# available_data_01 = {

#     "name": "csv_fish_all_data",
#     "type" : "csv",
#     "route": "data/catfishall.csv",
#     "query": "",
#     "key": key01,
#     "value": -1
# }
cont = """{
    'columns': []
    }
    'constrains': '',
    'descriptions': 'This dataframe is currently empty and needs to be created'
} """
requested_data = {
    "dataframe" : "Fish_all_data",
    "query": querystr,
    "file_path": "",
    "file_path_type" : "Pandas data frame",
    "content": cont,
    "value": ""
}


# key01 = """Fish capture monthly data containing years 2000 to 2012
# It contains only columns Date and Total but only for the selected years 
# The columns are: Date in the format MM/dd/yyyy and 
# Total which is a number. 
# The Date column is the primary key for this data set.
# """
cont1 = """{
    'columns': {
        ['columnname': 'Date',
        'format': datetime,
        'primarykey': 'yes'
        ],
        [
        'columnname': 'Total',
        'format': 'number',
        'primarykey': 'no'
        ]
    }
    'constrains': 'Years from 2000 to 2012',
    'descriptions': 'Contains only years 2000 to 2012 and only columns Date and Total' 
} """
available_data_01 = {

    "dataframe": "data_from_2000_on",
    "file_path": "data/catfich-from_2000.csv",
    "file_path_type" : "csv",   
    "query": "",
    "content": cont1,
    "value": -1
}

# key02 = """ Fish capture monthly data containing years 1986 to 1999. 
# It contains only columns Date and Total but only for the selected years 
# The columns are: Date in the format MM/dd/yyyy and  
# Total which is a number. 
# The Date column is the primary key for this data set.
# """
cont2 = """{
    'columns': {
        ['columnname': 'Date',
        'format': datetime,
        'primarykey': 'yes'
        ],
        [
        'columnname': 'Total',
        'format': 'number',
        'primarykey': 'no'
        ]
    }
    'constrains': 'Years from 1986 to 1999',
    'descriptions': 'Contains only years 1986 to 1999 and only columns Date and Total' 
} """

available_data_02 = {
    "dataframe" : "fish_from_1986_up_tp_1999",
    "file_path": "data/catfich-until1999.csv",
    "file_path_type" : "csv",   
    "query": "",
    "content": cont2,
    "value": -1
}

# key03 = """ Fish capture monthly data containing all years years 1986 to 2012 
# but it only contains column date and Wheight
# The columns are: Date in the format MM/dd/yyyy and
# Wheight which is a number. 
# The Date column is the primary key for this data set.
#"""
cont3 = """{
    'columns': {
        ['columnname': 'Date',
        'format': datetime,
        'primarykey': 'yes'
        ],
        [
        'columnname': 'Wheight',
        'format': 'number',
        'primarykey': 'no'
        ]
    }
    'constrains': 'Years from 1986 to 2012',
    'descriptions': 'Contains all the years but only columns Date and Wheight'
} """

available_data_03 = {
    "dataframe" : "fish_containing_wheight",
    "file_path": "data/catfish-Allyears-onlywheight.csv",
    "file_path_type" : "csv",   
    "query": "",
    "content": cont3,
    "value": -1
}

metadata_lst = [    
    json.dumps(requested_data),
    json.dumps(available_data_01),
    json.dumps(available_data_02),
    json.dumps(available_data_03)
]

found_metadata_list = []

load_dotenv("C:\Projrcts\Python\MSJupiter\myenv.env")
api_key = os.getenv("API_KEY")
openai.api_key = api_key

In [ ]:
sys = """ You are a good data analyst assistant and have a good understanding 
about tables, dataframes, csv and data structure in general
"""
my_string = " ".join(metadata_lst)
usr = " You have to consider the elements in the following list " + my_string + """ 
 The first element of the list is the dataframe that we want to construct, the others  
 data sets are the ones that could be used to build the the first one. You have to consider the content of
 each  data set to determine if it is relevant or not to build the dataframe specified in the first
  element of the list"""

assist = " Consider the following query: " + querystr + """ Use this query to find the provided
 list which is the most relevant to build the dataframe specified in the query. For that compare eqch element 
 of the list comparing the content attribute with the provided query and the content value of the first element of the list.
 Calculate a value between 0 and 5 where 5 is the most valued one, for this compare the content attribute 
 of the first dataframe with the rest and assign more value to the one providing more new data. Provide me the list of the dataframe 
 and value attributes of each element. Remember the first element is the one we want to build so the values have to be
 calculated for the second element on. Please provide the result in a Json format. The Json should look like this, do not add any additional explanation:
 [
  {"dataframe": 'The dataframe name',"file_path":'the file_path',"file_path_type":'file_path_type', "content": "the content attribute" "value": ?}, {...}
 ]
 Do not provide any aditional explanation, Only the Json object that I requested.
"""
#print(usr)
response = get_response_from_gpt4(sys, usr, assist)
response_message = response["choices"][0]["message"]
cont = response_message["content"]
result_table = json.loads(str(cont))
ref1 = json.dumps(requested_data)

max_element_df_name = max(result_table, key=lambda x: x['value'] if x['dataframe'] not in found_metadata_list else float('-inf'))

#max_element = json.dumps(max(result_table, key=lambda x: x['value']))
max_element = json.dumps(max_element_df_name)
max_df = max_element_df_name["dataframe"]
found_metadata_list.append(max_df)
#print(cont)

# Construct the Python function 
sys1 = """You are an accomplished data analyst assistant with the ability to write Python scripts for data 
manipulation tasks. You are expected to use various sources of data to gradually build the requested 
final dataframe described in the query. It's important to note that the result does not need to be 
achieved in a single step. Instead, you should focus on incrementally constructing the resulting 
dataframe based on the available data. Avoid creating elements or adding columns/rows that are not 
provided in the data or NaN. The goal is to build the dataframe in accordance with the query, ensuring that 
the content aligns with the available data sources. Overall, you are expected to leverage your skills 
and expertise in data analysis and utilize pandas to read CSV files, perform various dataframe 
operations, and incrementally construct the desired dataframe based on the query specifications and 
the available data. 
"""
assist1 = """ You will receive a query and two metadata references describing two different dataframes,the 
first is the resulting dataframe and the second is the one providing additional data to be included 
into the first one. The query contains the expected final result, and the references has the following 
attributes: dataframe: the name of the dataframe to be created, query: the description of the expected 
final result, file_path: the name of the file to be read into a dataframe, the file_path_type: 
typically csv, content: describing the content of that dataframe more specifically the columns of the dataframe and value: you can ignore this attribute. 
The first reference 'reference1' is the description of the resulting dataframe that you need to build, 
the second 'reference2' is the reference to the dataframe that will provide new data to augment the 
dataframe described in 'reference1'.The task at hand involves the creation of a Python script that 
incorporates the data from the 'reference2' dataframe into the resulting dataframe described in 
'reference1'. This requires implementing dataframe operations to combine the two dataframes, 
ensuring that the resulting dataframe reflects the changes made after incorporating the data from 
'reference2'.To accomplish this, the Python script needs to perform the necessary dataframe operations, 
like join, concat and others, never use'append' nor merge. Use join every time you need to create or update a column based on a common key column 
considering the content attribute of 'reference2' and the expected final result specified in the query 
attribute of 'reference1', very important is to know which columns contain each dataset. These operations should be designed to integrate the data from 'reference2' 
into the resulting dataframe and generate a new content that accurately reflects the changes. It is 
crucial to pay attention to the provided instructions and the content of each dataframe and ensure that the resulting dataframe adheres 
to the required structure and contains the expected columns and data for every step
The Python script should be crafted meticulously, incorporating the necessary dataframe operations to 
combine the dataframes, update the resulting dataframe, and generate the desired final output.In summary, 
the creation of the Python script must involve thoughtful dataframe operations to incorporate the data 
from 'reference2' into the resulting dataframe described in 'reference1'. The script should generate a 
new content for 'reference1' that accurately reflects the changes made after integrating the data from 
'reference2' This new content must be in the json format  like the one in referenc1 and reference2. 
Do not include any Python comments in the Python script you create.
"""

usr1 = "Use the query and the two references to generate a Python script to oftain a resultinng dataset and the  new content for that dataset; query: " + querystr + ", reference1: " + ref1 + ", reference2: " + max_element 
usr1 = usr1 + """. You are going to do an operation of the type df1 = P(df1, df2) where df1 is the dataframe described in ‘reference1’, df2 is the dataframe described in ‘reference2’ and P is the Python script you need to generate to augment df1 with the content of df2. The type of dataframe operations in P will depend on the ‘content’ attribute in ‘reference1’ and reference2’ and the query, following there are a group of operations you want to consider: create db1 when 'file_path’ is empty. Read db1 and db2 from 'file_path’ if it is not empty. Perform an outer join if there are no common columns between db1 and db2 other than the primary key in this case you don’t need to use lsuffix, rsuffix and combine_first. Perform inner, left, or right join when there are common columns and then do combine_first to reduce columns generated with lsuffix and rsuffix to the proper column name as described in the query. Only use valid Pandas dataframe operations, append is not a valid one. At the end of the script, you need to save df1 into a csv file with name equals to the ‘dataframe’ attribute of ‘reference1’. Do not include comments, using '#' in the generated Python code, only Python commands. 
"""

func = [
            {
                "name": "execute_python",
                "description": "Execute the Python code generated by GPT",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "pythonscript": {
                            "type": "string",
                            "description": "The Python script containing the computation and workflow generated by GPT when no creation or join is necessary. The last part of the script is a renaiming of the columns to make it compatible with the request.",
                        },
                        "newcontent": {
                            "type": "string",
                            "description": "The new content for the resulting dataframe. Is the content of reference1 augmented withe the content of reference2. ",
                        }
                    },
                    "required": ["pythonscript", "newcontent"],
                },
            },
            {
                "name": "execute_operation_between_rederence1_and_reference2",
                "description": "execute the Python code that reads reference1 dataframe and augmente it with the dataframe in reference2",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "pythonscript": {
                            "type": "string",
                            "description": "The Python script containing the computation and workflow generated by GPT. Start by reading the csv file containing df1 and df2.",
                        },
                        "newcontent": {
                            "type": "string",
                            "description": "The new content for the resulting dataframe. Is the content of reference1 augmented withe the content of reference2. ",
                        }
                    },
                    "required": ["execute_dataframe_left_join", "newcontent"],
                },
            },
            {
                "name": "execute_create_generate_first_time_from_reference2",
                "description": "Execute the Python code to generate, create and populate, the result dataframe, only if 'file_path' is empty, directly copying the one described in reference2  and not generating columns not  or renaming columns artificially",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "pythonscript": {
                            "type": "string",
                            "description": "The Python script containing the computation and workflow generated by GPT. The last part of the script is a renaiming of the columns to make it compatible with the request.",
                        },
                        "newcontent": {
                            "type": "string",
                            "description": "The new content for the resulting dataframe. Is the content of reference1 augmented withe the content of reference2. ",
                        }
                    },
                    "required": ["execute_dataframe_create_result_first_time", "newcontent"],
                }
            }            
        ]  
response1 = get_response_from_gpt4_funct(sys1, usr1, assist1, func)
response_message = response1["choices"][0]["message"]

if response_message.get("function_call"):
    function_name = response_message["function_call"]["name"]
    print("function_name:", function_name)
    #text = response_message["function_call"]["arguments"]
    #print(text)
    function_args = json.loads(response_message["function_call"]["arguments"])
    # Write the file with the generated script
    text = function_args['pythonscript']
    newcont = function_args['newcontent']
    lines = text.split('\n')
    # Write the lines to the file
    with open('data/add_numbers.py', 'w') as file:
        for line in lines:
            file.write(line + '\n')
        
    
    requested_data["file_path"] = "Fish_all_data.csv"
    requested_data["file_path_type"] = "csv"
    requested_data["content"] = newcont
    metadata_lst = [    
        json.dumps(requested_data),
        json.dumps(available_data_01),
        json.dumps(available_data_02),
        json.dumps(available_data_03)
]
    print(function_args)


In [14]:
import pandas as pd

# Read the dataframes from the csv files
df1 = pd.read_csv('Fish_all_data.csv')
df2 = pd.read_csv('data/catfich-until1999.csv')

# Ensure that 'Date' is of datetime type in both dataframes
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

# Perform an outer join on the 'Date' column
df1 = df1.set_index('Date').join(df2.set_index('Date'), how='outer', lsuffix='_df1', rsuffix='_df2')

# Combine the 'Total' columns from both dataframes
df1['Total'] = df1['Total_df1'].combine_first(df1['Total_df2'])

# Drop the unnecessary columns
df1 = df1.drop(columns=['Total_df1', 'Total_df2'])

# Save the resulting dataframe to a csv file
df1.to_csv('Fish_all_data.csv')

print(res)

KeyboardInterrupt: 